In [ ]:
import gensim
import pandas as pd
import time

In [ ]:
data_cleaned = pd.read_csv("data/data_cleaned.csv").fillna("")
data_cleaned

In [ ]:
train_array_token = [line.split(" ") for line in data_cleaned.Description.values]
print(train_array_token[:5])

In [ ]:
Features_dimension = 300
hs = 0
negative = 10

In [ ]:
sg = 1
print("Start learning skip-gram Word2Vec")
ts = time.time()
model_sg = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)


sg = 0
print("Start learning CBOW Word2Vec")
ts = time.time()
model_cbow = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)

In [ ]:
model_sg.most_similar(["xbox"])

In [ ]:
model_cbow.most_similar(["xbox"])

In [ ]:
model_sg["xbox"]

# Extra Work

pre-trained network


Github du projet[https://github.com/Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors) appris sur 1Giga d'articles de wikipedia en mode **Skip-Gram*

Vous pouvez télécharger ce modèle en suivant ce [lien](https://drive.google.com/file/d/0B0ZXk88koS2KM0pVTktxdG15TkE/view). Dezipez-le puis téléchargez le modèle en indiquant la direction du fichier "fr/bin"

In [ ]:
model_online_dir = "data/fr/fr.bin"
#model_online_dir = "ACOMPLETER/fr.bin"
model_online = gensim.models.Word2Vec.load(model_online_dir)